# Data Sourcing

In [2]:
import pkg_resources
import pip
installedPackages = {pkg.key for pkg in pkg_resources.working_set}
required = {'researchpy', 'missingno', 'folium', 'pydotplus','bokeh','imblearn', 'catboost'}
missing = required - installedPackages
if missing:
    !pip install researchpy
    !pip install missingno
    !pip install folium
    !pip install pydotplus
    !pip install bokeh
    !pip install imblearn
    !pip install catboost
    #!pip install xgboost
    #!pip install graphviz

In [1]:
#Disable the warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
import timeit
start_time = timeit.default_timer()  #timestamp to calculate total runtime

import pandas as pd
import numpy as np

import researchpy as rp
import missingno as msno
import itertools
import scipy.stats as ss

import seaborn as sns
import matplotlib.pyplot as plt
import folium
from folium import plugins
import graphviz

from sklearn import tree
from sklearn import feature_selection
from sklearn import preprocessing

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

from collections import Counter

from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

from sklearn.utils import resample
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, accuracy_score, \
precision_score, recall_score, roc_auc_score, f1_score, precision_recall_curve, auc 

%matplotlib inline
pd.set_option('display.max_columns', None)  # prevent column output trancation
sns.set()  # change plot styling from Matlab's 90s feel to today's Seaborn.

In [3]:
# File Directories
path_crashes = 'data/crashes.sample2020.csv'
path_vehicles = 'data/vehicles.sample.csv'
path_people = 'data/people.sample.csv'

# Import samples
crashes = pd.read_csv(path_crashes, parse_dates=["CRASH_DATE", "CRASH_DATE_EST_I", "DATE_POLICE_NOTIFIED"],
                      low_memory=False, dtype=object)
vehicles = pd.read_csv(path_vehicles, parse_dates=["CRASH_DATE"], low_memory=False, dtype=object)
people = pd.read_csv(path_people, parse_dates=["CRASH_DATE"], low_memory=False, dtype=object)

In [6]:
# Joining datasets
non_passengers=people[people.PERSON_ID.str.contains('^O')]

vehicles_with_people=vehicles.merge(non_passengers,how='left',on=['CRASH_RECORD_ID','RD_NO','CRASH_DATE','VEHICLE_ID'])

data=crashes.merge(vehicles_with_people,how='inner',on=['CRASH_RECORD_ID','RD_NO','CRASH_DATE'])

# Feature Selection
filter_list=["AGE","LANE_CNT","AIRBAG_DEPLOYED","PRIM_CONTRIBUTORY_CAUSE","POSTED_SPEED_LIMIT","NUM_UNITS","TRAFFICWAY_TYPE",  
             "SEC_CONTRIBUTORY_CAUSE","FIRST_CRASH_TYPE","MOST_SEVERE_INJURY","LIGHTING_CONDITION","SEX","CRASH_DATE",
             "CRASH_HOUR","VEHICLE_YEAR"]

# Data that will be used in predictions
modeling_data=data[filter_list]

In [7]:
# Split modeling data into raw train and test sets
raw_train, raw_test = train_test_split(modeling_data, test_size=0.20, random_state=42, shuffle=True)

# Data Preprocessing

## Preprocessing Train Set

In [8]:
def preprocessor(dataframe):
    '''Preprocesses df and returns X and y ready for modeling (after imputation of numericals!)'''
    df = dataframe.copy()
    
    # Prepare data for missing value imputation
    df.loc[df["LIGHTING_CONDITION"] == "UNKNOWN", "LIGHTING_CONDITION"] = np.nan
    df.loc[df["TRAFFICWAY_TYPE"] == "UNKNOWN","TRAFFICWAY_TYPE"] = np.nan
    df.loc[df["AIRBAG_DEPLOYED"] == "DEPLOYMENT UNKNOWN","AIRBAG_DEPLOYED"] = np.nan
    df.fillna({'LIGHTING_CONDITION': 'DAYLIGHT', 'TRAFFICWAY_TYPE': 'NOT DIVIDED',
               'SEX': 'UNABLE TO DETERMINE', 'AIRBAG_DEPLOYED': 'UNABLE TO DETERMINE'}, inplace=True)
    
    # Remove rows missing most severe injury results
    drop_rows = ['MOST_SEVERE_INJURY']
    df.dropna(how ='any', subset = drop_rows, inplace = True)
    
    # Handle numerical features
    df['VEHICLE_YEAR'] = pd.to_numeric(df['VEHICLE_YEAR'])
    df['NUM_UNITS'] = pd.to_numeric(df['NUM_UNITS'])
    df["POSTED_SPEED_LIMIT"] = pd.to_numeric(df["POSTED_SPEED_LIMIT"])
    df["AGE"] = pd.to_numeric(df["AGE"])
    
    df['LANE_CNT'] = pd.to_numeric(df['LANE_CNT'])    
    df['LANE_CNT'].fillna(2, inplace=True)
    df.loc[(df['LANE_CNT'] > 6),'LANE_CNT'] = 6
    
    # Function definitions
    def injury(x): 
        if any(s in x for s in ["FATAL","NONINCAPACITATING INJURY","INCAPACITATING INJURY"]):
            return "INJURED"
        else:
            return "NOT INJURED"
    
    def airbag(x):
        if ("DEPLOY" in x) and ("UNKNOWN" not in x):
            if "NOT" in x:
                return "NOT DEPLOYED"
            else:
                return "DEPLOYED"
        else:
            return x

    def crash_hour(x):
        if  2 <= x < 8:
            return "Early_morning"
        elif 8 <= x < 12:
            return "Morning"
        elif 12 <= x < 18:
            return "Afternoon"
        else:
            return "Night"
  
    def traffic_way(x):
        if ("NOT" in x) or ("ONE-WAY" in x):
            return "NOT_DIVIDED"
        else:
            return "DIVIDED"
    
    # Feature Engineering
    df["INJURY"] = df["MOST_SEVERE_INJURY"].apply(lambda x: injury(x))
    df["AIRBAG_DEPLOYED"] = df["AIRBAG_DEPLOYED"].apply(lambda x: airbag(x))
    df["CRASH_HOUR"] = df["CRASH_HOUR"].apply(lambda x: crash_hour(int(x)))
    df["TRAFFICWAY_TYPE"] = df["TRAFFICWAY_TYPE"].apply(lambda x: traffic_way(x))
    
    df["VEHICLE_AGE"] = df["CRASH_DATE"].dt.year-df["VEHICLE_YEAR"]
    df.loc[df["VEHICLE_AGE"] < 0, "VEHICLE_AGE"] = 0
    df.drop(["VEHICLE_YEAR", "CRASH_DATE", "MOST_SEVERE_INJURY"], axis=1, inplace=True)
    
    # Splitting df into X and y
    y = df["INJURY"]
    X = df.drop(["INJURY"], axis=1)
    
    # Binarize y
    from sklearn.preprocessing import label_binarize
    y = preprocessing.label_binarize(y, classes=['NOT INJURED', 'INJURED'])
    
    # One-Hot Encoding
    X = pd.get_dummies(X, columns = X.select_dtypes(['object']).columns)
    dummies_to_drop = X.columns[X.columns.str.contains("UNABLE|UNKNOWN|NOT APPLICABLE|OTHER")]
    X = X.loc[:, ~X.columns.isin(dummies_to_drop)]
      
    return(X, y)

In [9]:
# Preprocess train set
X_train, y_train = preprocessor(raw_train)

In [ ]:
# Fit imputer and impute missing values on train set 
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors = 5)
df = pd.DataFrame(imputer.fit_transform(X_train), columns = X_train.columns)
X_train = df.copy()
X_train = np.round(X_train)

In [ ]:
X_train.isna().any().sum()

## Preprocessing Test Set

In [ ]:
# Preprocess test set
X_test, y_test = preprocessor(raw_test)

In [ ]:
# Add two trivial features of train set that doesn't exist in test set in order to use pre-fit imputer
X_test["PRIM_CONTRIBUTORY_CAUSE_OBSTRUCTED CROSSWALKS"] = 0
X_test["SEC_CONTRIBUTORY_CAUSE_PASSING STOPPED SCHOOL BUS"] = 0
X_test = X_test[X_train.columns]

In [ ]:
# Impute missing values on test set by pre-fit imputer object 
df = pd.DataFrame(imputer.transform(X_test), columns = X_test.columns)
X_test = df.copy()
X_test = np.round(X_test)

In [ ]:
X_test.isna().any().sum()

## Preprocessing EDA Train Set

In [13]:
def eda_preprocessor(dataframe):
    '''Preprocesses and returns the entire train set for EDA'''
    
    df = dataframe.copy()
    
    # Split data into train test with exact indexing
    train, test = train_test_split(df, test_size=0.20, random_state=42, shuffle=True)
    
    # Feature reduction
    drop=["CRASH_DATE_EST_I", "REPORT_TYPE", "CRASH_DATE_EST_I", "REPORT_TYPE", "DATE_POLICE_NOTIFIED",
          "BEAT_OF_OCCURRENCE", "PHOTOS_TAKEN_I", "STATEMENTS_TAKEN_I", "WORK_ZONE_TYPE", "WORKERS_PRESENT_I", 
          "INJURIES_NO_INDICATION", "INJURIES_UNKNOWN", "CRASH_DAY_OF_WEEK", "CRASH_MONTH", "RD_NO", "VEHICLE_ID", 
          "CRASH_RECORD_ID", "SEAT_NO", "STATE", "ZIPCODE", "DRIVERS_LICENSE_STATE", "PERSON_ID", "DRIVERS_LICENSE_CLASS", 
          "INJURY_CLASSIFICATION", "HOSPITAL", "EMS_AGENCY", "EMS_RUN_NO", "PEDPEDAL_LOCATION", "LOCATION",
          "DAMAGE", "CRASH_TYPE", "MODEL", "BAC_RESULT", "CRASH_UNIT_ID", "RD_NO", "UNIT_NO", "UNIT_TYPE", 
          "VEHICLE_ID", "CMRC_VEH_I", "MAKE", "TOWED_I", "FIRE_I", "TOWED_BY", "STREET_NO","STREET_NAME",
          "TOWED_TO", "AREA_00_I", "AREA_01_I", "AREA_02_I", "AREA_03_I", "AREA_04_I", "AREA_05_I", "AREA_06_I", 
          "AREA_07_I", "AREA_08_I", "AREA_09_I", "AREA_10_I", "AREA_11_I", "AREA_12_I", "AREA_99_I", "CMV_ID", 
          "USDOT_NO", "CCMC_NO", "ILCC_NO", "COMMERCIAL_SRC", "GVWR", "CARRIER_NAME", "CARRIER_STATE", 
          "CARRIER_CITY", "HAZMAT_PLACARDS_I", "HAZMAT_NAME", "UN_NO", "HAZMAT_PRESENT_I", "HAZMAT_REPORT_I",
          "HAZMAT_REPORT_NO", "MCS_REPORT_I", "MCS_REPORT_NO", "HAZMAT_VIO_CAUSE_CRASH_I", "MCS_VIO_CAUSE_CRASH_I", 
          "IDOT_PERMIT_NO", "WIDE_LOAD_I", "TRAILER1_WIDTH", "TRAILER2_WIDTH", "TRAILER1_LENGTH", "TRAILER2_LENGTH", 
          "TOTAL_VEHICLE_LENGTH", "AXLE_CNT", "VEHICLE_CONFIG", "CARGO_BODY_TYPE", "LOAD_TYPE", "HAZMAT_OUT_OF_SERVICE_I",
          "INJURIES_REPORTED_NOT_EVIDENT", "INJURIES_NON_INCAPACITATING", #"INJURIES_FATAL","INJURIES_INCAPACITATING",
          "MCS_OUT_OF_SERVICE_I", "HAZMAT_CLASS"]
    
    train = train.loc[:, ~train.columns.isin(drop)]
    
    
    # Prepare data for missing value imputation
    train.loc[train["TRAFFIC_CONTROL_DEVICE"]=="UNKNOWN","TRAFFIC_CONTROL_DEVICE"]=np.nan
    train.loc[train["DEVICE_CONDITION"]=="UNKNOWN","DEVICE_CONDITION"]=np.nan
    train.loc[train["WEATHER_CONDITION"]=="UNKNOWN","WEATHER_CONDITION"]=np.nan
    train.loc[train["LIGHTING_CONDITION"]=="UNKNOWN","LIGHTING_CONDITION"]=np.nan
    train.loc[train["TRAFFICWAY_TYPE"]=="UNKNOWN","TRAFFICWAY_TYPE"]=np.nan
    train.loc[train["ROADWAY_SURFACE_COND"]=="UNKNOWN","ROADWAY_SURFACE_COND"]=np.nan
    train.loc[train["ROAD_DEFECT"]=="UNKNOWN","ROAD_DEFECT"]=np.nan

    train.loc[train["VEHICLE_DEFECT"]=="UNKNOWN","VEHICLE_DEFECT"]=np.nan
    train.loc[train["VEHICLE_TYPE"]=="UNKNOWN/NA","VEHICLE_TYPE"]=np.nan
    train.loc[train["TRAVEL_DIRECTION"]=="UNKNOWN","TRAVEL_DIRECTION"]=np.nan
    train.loc[train["MANEUVER"]=="UNKNOWN/NA","MANEUVER"]=np.nan

    train.loc[train["SAFETY_EQUIPMENT"]=="USAGE UNKNOWN","SAFETY_EQUIPMENT"]=np.nan
    train.loc[train["AIRBAG_DEPLOYED"]=="DEPLOYMENT UNKNOWN","AIRBAG_DEPLOYED"]=np.nan
    train.loc[train["EJECTION"]=="UNKNOWN","EJECTION"]=np.nan
    train.loc[train["DRIVER_ACTION"]=="UNKNOWN","DRIVER_ACTION"]=np.nan
    train.loc[train["DRIVER_VISION"]=="UNKNOWN","DRIVER_VISION"]=np.nan
    train.loc[train["PHYSICAL_CONDITION"]=="UNKNOWN","PHYSICAL_CONDITION"]=np.nan
    train.loc[train["PEDPEDAL_ACTION"]=="UNKNOWN/NA","PEDPEDAL_ACTION"]=np.nan
    
    
    # Mıssing Value Imputation
    train.fillna({
        'TRAFFIC_CONTROL_DEVICE': 'NO CONTROLS',
        'DEVICE_CONDITION': 'NO CONTROLS',
        'WEATHER_CONDITION': 'CLEAR',
        'LIGHTING_CONDITION': 'DAYLIGHT', # ??
        'TRAFFICWAY_TYPE': 'NOT DIVIDED', # ??
        'ROADWAY_SURFACE_COND': 'NO DEFECTS',
        'ROAD_DEFECT': 'CLEAR',
        'INTERSECTION_RELATED_I': 'N',
    
        'NOT_RIGHT_OF_WAY_I': 'N',
        'HIT_AND_RUN_I': 'N',
        'DOORING_I': 'N',
        'WORK_ZONE_I': 'N',
        'NUM_PASSENGERS': 0,
        'LIC_PLATE_STATE': 'IL',
        'VEHICLE_DEFECT': 'UNABLE TO DETERMINE',
        'VEHICLE_TYPE': 'OTHER',

        'VEHICLE_USE': 'OTHER',
        'TRAVEL_DIRECTION': 'N',
        'MANEUVER': 'OTHER',
        'OCCUPANT_CNT': 0,
        'EXCEED_SPEED_LIMIT_I': 'N',
        'FIRST_CONTACT_POINT': 'OTHER',
        'PERSON_TYPE': 'UNABLE TO DETERMINE',
        'CITY': 'OTHER',

        'SEX': 'UNABLE TO DETERMINE',
        'AIRBAG_DEPLOYED': 'UNABLE TO DETERMINE',
        'EJECTION': 'UNABLE TO DETERMINE',
        'DRIVER_ACTION': 'OTHER',
        'DRIVER_VISION': 'OTHER',
        'PHYSICAL_CONDITION': 'UNABLE TO DETERMINE',
        'PEDPEDAL_ACTION': 'UNABLE TO DETERMINE',
        'PEDPEDAL_VISIBILITY': 'UNABLE TO DETERMINE',

        'CELL_PHONE_USE': 'UNABLE TO DETERMINE',
        'SAFETY_EQUIPMENT': 'UNABLE TO DETERMINE',
        'BAC_RESULT VALUE': 0
    }, inplace=True)
    
    
    # Drop rows with missing values in these features
    drop_rows = ['INJURIES_TOTAL', 'LATITUDE', 'MOST_SEVERE_INJURY']
    train.dropna(how = 'any', subset = drop_rows, inplace = True)
    

    # Handle numerical features
    train['LANE_CNT'] = pd.to_numeric(train['LANE_CNT'])
    train['VEHICLE_YEAR'] = pd.to_numeric(train['VEHICLE_YEAR'])
    train['NUM_UNITS'] = pd.to_numeric(train['NUM_UNITS'])
    train["POSTED_SPEED_LIMIT"] = pd.to_numeric(train["POSTED_SPEED_LIMIT"])
    train["AGE"] = pd.to_numeric(train["AGE"])
    train["INJURIES_FATAL"] = pd.to_numeric(train["INJURIES_FATAL"])
    train["INJURIES_INCAPACITATING"] = pd.to_numeric(train["INJURIES_INCAPACITATING"])
    train["INJURIES_TOTAL"] = pd.to_numeric(train["INJURIES_TOTAL"])
    
    train['TOTAL_FATAL'] = train['INJURIES_FATAL'] + train['INJURIES_INCAPACITATING']
    train['TOTAL_FATAL'] = pd.to_numeric(train['TOTAL_FATAL'])

    train['LANE_CNT'] = pd.to_numeric(train['LANE_CNT'])    
    train['LANE_CNT'].fillna(2, inplace=True)
    train.loc[(train['LANE_CNT'] > 6),'LANE_CNT'] = 6
    
    
    # Function definitions
    def injury(x): 
        if any(s in x for s in ["FATAL","NONINCAPACITATING INJURY","INCAPACITATING INJURY"]):
            return "INJURED"
        else:
            return "NOT INJURED"
        
    def fatalities(x): 
        if any(s in x for s in ["FATAL"]):
            return "FATAL"
        else:
            return "NOT FATAL"
    
    def airbag(x):
        if ("DEPLOY" in x) and ("UNKNOWN" not in x):
            if "NOT" in x:
                return "NOT DEPLOYED"
            else:
                return "DEPLOYED"
        else:
            return x
  
    def traffic_way(x):
        if ("NOT" in x) or ("ONE-WAY" in x):
            return "NOT_DIVIDED"
        else:
            return "DIVIDED"
        
    def contact_point(x):
        if "FRONT" in x:
            return "FRONT"
        elif "SIDE" in x:
            return "SIDE"
        elif "REAR" in x:
            return "REAR"
        else:
            return "OTHER"
    
    def equip_used(x):
        if ("USED" in x) or ("HELMET" in x) or ("NONE PRESENT" in x):
            if any(s in x for s in ["NOT","IMPROPER","NONE PRESENT"]):
                return "DID NOT USE SAFETY EQUIP"
            else:
                return "USED SAFETY EQUIP"
        else:
            return x

    def airbag(x):
        if ("DEPLOY" in x) and ("UNKNOWN" not in x):
            if "NOT" in x:
                return "NOT DEPLOYED"
            else:
                return "DEPLOYED"
        else:
            return x
    
    def crash_hour(x):
        if  2 <= x < 8:
            return "Early_morning"
        elif 8 <= x < 12:
            return "Morning"
        elif 12 <= x < 18:
            return "Afternoon"
        else:
            return "Night"

    def traffic_control(x):
        if ("NO CONTROLS" in x) or ("UNKNOWN" in x) or ("OTHER" in x):
            return "NO_SIGN"
        else:
            return "SIGN"
    
    def location(x1,x2):
        if (41.84 <= float(x1) <= 41.9100064) and (-87.7421459 <= float(x2) <= -87.50):
            return "Downtown"
        else:
            return "Not Downtown"

        
    # Feature engineering
    train["INJURY"] = train["MOST_SEVERE_INJURY"].apply(lambda x: injury(x))
    train["FATALITIES"] = train["MOST_SEVERE_INJURY"].apply(lambda x: fatalities(x))
    train["AIRBAG_DEPLOYED"] = train["AIRBAG_DEPLOYED"].apply(lambda x: airbag(x))
    train["CRASH_HOUR"] = train["CRASH_HOUR"].apply(lambda x: crash_hour(int(x)))
    train["TRAFFICWAY_TYPE"] = train["TRAFFICWAY_TYPE"].apply(lambda x: traffic_way(x))
    train["FIRST_CONTACT_POINT"] = train["FIRST_CONTACT_POINT"].apply(lambda x: contact_point(x))
    train["MANEUVER"] = train["MANEUVER"].apply(lambda x: "TURN" if "TURN" in x else("LANE" if any(s in x for s in ["LANE","OVER","ENTER"]) else x))
    train["MANEUVER"] = train["MANEUVER"].apply(lambda x: "OTHER" if all(s not in x for s in ["AHEAD","TURN","UNKNOWN","LANE","BACKING"]) else x)
    train["SAFETY_EQUIPMENT"] = train["SAFETY_EQUIPMENT"].apply(lambda x: equip_used(x))
    train["AIRBAG_DEPLOYED"] = train["AIRBAG_DEPLOYED"].apply(lambda x: airbag(x))
    train["TRAFFIC_CONTROL_DEVICE"]= train["TRAFFIC_CONTROL_DEVICE"].apply(lambda x: traffic_control(x))
    train["Location"] = train.apply(lambda x: location(x["LATITUDE"], x["LONGITUDE"]), axis = 1)
    
    train["VEHICLE_AGE"] = train["CRASH_DATE"].dt.year-train["VEHICLE_YEAR"]
    train.loc[train["VEHICLE_AGE"] < 0, "VEHICLE_AGE"] = 0
    train.drop(["VEHICLE_YEAR", "MOST_SEVERE_INJURY"], axis=1, inplace=True)  # "CRASH_DATE" for Tony
     
    # Return only train set for EDA purposes     
    return(train)

In [14]:
train = eda_preprocessor(data)

In [20]:
train.INJURY.value_counts(normalize=True)

NOT INJURED    0.891667
INJURED        0.108333
Name: INJURY, dtype: float64

# Comparison of Imputation Methods

In [ ]:
# To use the experimental IterativeImputer, we need to explicitly ask for it:
from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.impute import SimpleImputer, KNNImputer, IterativeImputer
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline

In [ ]:
X_train_imp, y_train_imp = preprocessor(raw_train)

random_final = RandomForestClassifier(n_estimators=1000, min_samples_split=15, min_samples_leaf=5, 
                                      max_features='log2', max_depth=13, criterion='entropy', random_state=42)

In [ ]:
def get_scores_for_imputer(imputer, X_missing, y_missing, scoring='recall', cv=5):
    estimator = make_pipeline(imputer, random_final)
    impute_scores = cross_val_score(estimator, X_missing, y_missing, scoring=scoring, cv=cv)
    return np.mean(impute_scores), np.std(impute_scores)

In [ ]:
imputer_recall_mean = np.zeros(3)
imputer_recall_std = np.zeros(3)

#imputer_recall_mean[0], imputer_recall_std[0] = get_scores_for_imputer(X_train_imp, y_train_imp)

In [ ]:
def get_impute_median_score(X_missing, y_missing, scoring='recall', cv=5):
    imputer = SimpleImputer(missing_values=np.nan, strategy="median", add_indicator=True)
    median_impute_scores = get_scores_for_imputer(imputer, X_missing, y_missing, scoring=scoring, cv=cv)
    return np.mean(median_impute_scores), np.std(median_impute_scores)

imputer_recall_mean[0], imputer_recall_std[0] = get_impute_median_score(X_train_imp, y_train_imp)

In [ ]:
def get_impute_knn_score(X_missing, y_missing, scoring='recall', cv=5):
    imputer = KNNImputer(missing_values=np.nan, add_indicator=True)
    knn_impute_scores = get_scores_for_imputer(imputer, X_missing, y_missing, scoring=scoring, cv=cv)
    return np.mean(knn_impute_scores), np.std(knn_impute_scores)

imputer_recall_mean[1], imputer_recall_std[1] = get_impute_knn_score(X_train_imp, y_train_imp)

In [ ]:
def get_impute_iterative_score(X_missing, y_missing, scoring='recall', cv=5):
    imputer = IterativeImputer(missing_values=np.nan, add_indicator=True, random_state=0, 
                               n_nearest_features=5, sample_posterior=True)
    iterative_impute_scores = get_scores_for_imputer(imputer, X_missing, y_missing, scoring=scoring, cv=cv)
    return np.mean(iterative_impute_scores), np.std(iterative_impute_scores)

imputer_recall_mean[2], imputer_recall_std[2] = get_impute_iterative_score(X_train_imp, y_train_imp)

In [ ]:
# Box-cox transformation (0=log)
#from scipy.stats import boxcox
#X_train_imp_np = np.round(X_train_imp)
#y_train_imp_np = np.round(y_train_imp)

#from scipy.special import inv_boxcox
#X_bc, fitted_lambda = ss.boxcox(np.round(X_train_imp.values), lmbda=None)
#X_bc = inv_boxcox(X_bc, fitted_lambda)

#y_bc, fitted_lambda = ss.boxcox(np.round(X_train_imp.values), lmbda=None)
#y_bc = inv_boxcox(y_bc, fitted_lambda)

#imputer_recall_mean[2], imputer_recall_std[2] = get_impute_iterative_score(X_bc, y_bc)

In [ ]:
imputer_recall_mean

In [ ]:
imputer_recall_std

In [ ]:
imputer_recall_mean_neg = imputer_recall_mean * -1

In [ ]:
x_labels = ['Median imputation', 'KNN Imputation', 'Iterative Imputation']
n_bars = len(imputer_recall_mean_neg)
xval = np.arange(n_bars)

colors = ['r', 'g', 'b', 'orange', 'black']

# plot results
plt.figure(figsize=(16, 8))
ax1 = plt.subplot(121)
for j in xval:
    ax1.barh(j, imputer_recall_mean_neg[j], #xerr=imputer_recall_std[j],
             color=colors[j], alpha=0.6, align='center')

ax1.set_title('Comparison of Imputation Techniques')
ax1.set_xlim(left = np.min(imputer_recall_mean_neg) * 0.9, right = np.max(imputer_recall_mean_neg) * 1.1)
ax1.set_yticks(xval)
ax1.set_xlabel('Recall')
ax1.invert_yaxis()
ax1.set_yticklabels(x_labels)

plt.show()

# Catboost

In [ ]:
drop_list = ["LATITUDE", "LONGITUDE", "INJURIES_TOTAL", "CRASH_DATE", "INJURY", "FATALITIES", "TOTAL_FATAL", 
             "INJURIES_INCAPACITATING", "INJURIES_INCAPACITATING", "INJURIES_FATAL"]
train_new = train.drop(drop_list, axis=1)
train_new = train_new[(train_new.PRIM_CONTRIBUTORY_CAUSE != 'UNABLE TO DETERMINE') & (train_new.PRIM_CONTRIBUTORY_CAUSE != 'NOT APPLICABLE')]

In [ ]:
X_train_cat = train_new.drop(["PRIM_CONTRIBUTORY_CAUSE"],axis=1).reset_index(drop=True)
y_train_cat = train_new["PRIM_CONTRIBUTORY_CAUSE"]
X_train_cat, X_val_cat, y_train_cat, y_val_cat = train_test_split(X_train_cat, y_train_cat, test_size=0.25, random_state=42, stratify=y_train_cat)

In [ ]:
categorical_features_indices = np.where(X_train_cat.dtypes == object)[0]

In [ ]:
from catboost import Pool, CatBoostClassifier

train_dataset = Pool(data = X_train_cat, label = y_train_cat, cat_features = categorical_features_indices)
eval_dataset = Pool(data=X_val_cat, label=y_val_cat, cat_features=categorical_features_indices)

# Initialize CatBoostClassifier
model = CatBoostClassifier(iterations=10, learning_rate=1, depth=2, loss_function='MultiClass')

In [ ]:
model.fit(train_dataset)

In [ ]:
y_pred = model.predict(eval_dataset)

# Print the confusion matrix
print(confusion_matrix(y_val_cat, y_pred))

# Print the precision and recall, among other metrics
print(classification_report(y_val_cat, y_pred, digits=3))

# Bootstrapped CIs for Model Results

In [ ]:
# Random Undersampling on X and y train sets
under = RandomUnderSampler(sampling_strategy=1)
X_under, y_under = under.fit_resample(X_train, y_train)

In [ ]:
# Store n_bootstraps number of bootstrap samples of X and y dataframes in lists
n_bootstraps = 1000

bootstrap_X_train = []
bootstrap_y_train = []
for _ in range(n_bootstraps):
    sample_X, sample_y = resample(X_under, y_under)
    bootstrap_X_train.append(sample_X)
    bootstrap_y_train.append(sample_y)
    
bootstrap_X_test = []
bootstrap_y_test = []
for _ in range(n_bootstraps):
    sample_X, sample_y = resample(X_test, y_test)
    bootstrap_X_test.append(sample_X)
    bootstrap_y_test.append(sample_y)

In [ ]:
# Random Forest model with tuned hyperparameters
random_final = RandomForestClassifier(n_estimators=1000, min_samples_split=15, min_samples_leaf=5, 
                                      max_features='log2', max_depth=13, criterion='entropy', random_state=0)

# Predict n_bootstraps number of bootstrapped samples and store recall results
recall_scores = []

for index, (train, test) in enumerate(zip(bootstrap_X_train, bootstrap_X_test)):
    model = random_final.fit(train, bootstrap_y_train[index])
    y_pred = model.predict(test)
    recall_scores.append(recall_score(bootstrap_y_test[index], y_pred))

Precision is how sure you are of your true positives while recall is how sure you are that you are not missing any positives.

Recall value of 0.70 means that 3 of every 10 injuried people in reality are missed by our model and 7 labeled as injuried.

In [ ]:
# Plot scores
plt.hist(recall_scores) # , bins=5 
plt.show()

# Confidence intervals
alpha = 0.95
p = ((1.0 - alpha) / 2.0) * 100
lower = max(0.0, np.percentile(recall_scores, p))
p = (alpha + ((1.0 - alpha) / 2.0)) * 100
upper = min(1.0, np.percentile(recall_scores, p))

print('%.1f confidence interval for recall %.1f%% and %.1f%% across %.d bootstrapped predictions.' % (alpha*100, lower*100, upper*100, n_bootstraps))

In [ ]:
#elapsed = timeit.default_timer() - start_time
#print('Total runtime is', round(elapsed, 2)/60, 'minutes.')

# Further Analysis

## Summary of Injuries Fatalities 


- **14% of fatalities** and **10% of injuries** happened to **people aged over 60**.
- **12% of fatalities** and **11% of injuries** happened in **vehicles aged over 15**.


- **25% of fatalities** and **13% of injuries** happened to **cyclers with no contrasting clothing**.
- **6% of injuries** happened to pedestrians or pedalists **while crossing the road**.
- **22% of fatalities** happened where drivers were **failed to reduce speed**.
- **22% of injuries** happened where drivers were **failed to yield right of way**.


- **8% of fatalities** happened where drivers were **too fast for conditions**.
- **16% of fatalities** and **35% of injuries** happened in which **cars were failed to deploy airbags**.
- **14% of fatalities** and **13% of injuries** happened to **drivers failed to use safety equipment**.


- **22% of fatalities** and **11% of injuries** happened to **pedestrians**.
- **5% of fatalities** and **1% of injuries** happened in **the excess of speed limits**.


- **55% of fatalities** and **33% of injuries** happened **at night accidents**.
- **11% of fatalities** and **16% of injuries** happened in **wet roads**.
- **39% of fatalities** and **47% of injuries** happened in **collisions straight ahead**.
- **5% of fatalities** and **12% of injuries** happened in **a turn maneuver**.

In [ ]:
train = eda_preprocessor(data)

In [ ]:
def injury_fatality_summarizer(col, df=train):
    x=df[df['INJURY']=='INJURED'][col].value_counts(normalize=True).reset_index(name='injury')
    y=df[df['FATALITIES']=='FATAL'][col].value_counts(normalize=True).reset_index(name='fatal')
    
    table=x.merge(y, how='inner', on=['index']).sort_values(['fatal','injury'], ascending=False)
    table=table.rename(columns={'index':col, 'injury':'PERCENTAGE OF INJURIES', 'fatal':'PERCENTAGE OF FATALITIES'})
    return(table[:12].round(2))

In [ ]:
from IPython.core import display as ICD

for col in train.columns:
    table = injury_fatality_summarizer(col)
    ICD.display(table)

## Time-Series Findings

**Overview Structure based on Time**

- There is **at least 7 traffic crashes** in Chicago **on a daily basis**.


- **At least 11 people** are injured in a traffic crash **every day**!


- **Everyday, at least 1 person dies or faces with fatal injuries** in a traffic crash in Chicago!


- **Death and serious injuries** from traffic crashes **increased by 8% by each year**!

In [ ]:
# Convert crash_data into time
train['CRASH_DATE'] = pd.to_datetime(train.CRASH_DATE)
train['CRASH_DATE_NEW'] = pd.to_datetime(train['CRASH_DATE']).dt.date
#train['CRASH_DATE_NEW']

In [ ]:
train_new = train.copy()
from datetime import datetime, date
#df[df['CRASH_DATE_NEW'] == date(2020, 4, 1)]

In [ ]:
train_new.groupby(['CRASH_DATE_NEW'])['TOTAL_FATAL'].sum().sort_values(ascending = True).mean()

In [ ]:
train_new.groupby(['CRASH_DATE_NEW'])['INJURIES_TOTAL'].sum().sort_values(ascending = True).mean()

In [ ]:
train_new['YEAR'] = train_new.CRASH_DATE.dt.year
train_new['MONTH'] = train_new.CRASH_DATE.dt.month
#train_new.YEAR.value_counts()

In [ ]:
#plot data for Injuries Total
fig, ax = plt.subplots(figsize=(15,7))
train_new.groupby(['YEAR'])['TOTAL_FATAL'].sum().plot(ax = ax)

#r = [1,3,5,7]
#names = ['2017', '2018', '2019', '2020']
plt.xticks(fontsize=14)
#plt.yticks(fontsize=14)
plt.xlabel('Year', fontsize=14)
plt.ylabel('Total Number of Serious Injuries and Fatalities', fontsize=14)
plt.title('Yearly Data for Serious Injuries and Fatalities', fontsize=20)

In [ ]:
#plot monthly data for Injuries Total
fig, ax = plt.subplots(figsize=(15,7))
train_new.groupby(['MONTH'])['INJURIES_TOTAL'].sum().plot(ax = ax)

r = [2,4,6,8,10,12]
names = ['February', 'April', 'June', 'August', 'October', 'December']
plt.xticks(r, names, fontsize=14)
plt.yticks(fontsize=14)
plt.ylabel('Total Number of Injuries and Fatalities', fontsize=14)
plt.title('Monthly Data for Total Injuries', fontsize=20)

## Fatal Traffic Crashes By Travel Choice

People in vehicles account for more than half of traffic crash fatalities each year

In [ ]:
df = train_new.groupby(['YEAR', 'PERSON_TYPE']).agg({'TOTAL_FATAL': 'sum'})
df

# Change: groupby state_office and divide by sum
df1 = df.groupby(level=0).apply(lambda x:100 * x / float(x.sum()))
df1.reset_index(inplace=True)
df1=df1[df1['PERSON_TYPE'].isin(['BICYCLE','DRIVER','PEDESTRIAN'])]

In [ ]:
df1

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

fig=plt.figure(figsize = (14,8))
ax=fig.add_subplot(111)

x = ['2017', '2018', '2019', '2020']
ax.plot(x,df1[df1['PERSON_TYPE']=='DRIVER']['TOTAL_FATAL'],c='g',marker=(8,2,0),ls='--', linewidth=2,label='DRIVER')
ax.plot(x,df1[df1['PERSON_TYPE']=='BICYCLE']['TOTAL_FATAL'],c='r',marker="v",ls='-', linewidth=2,label='BICYCLE')
ax.plot(x,df1[df1['PERSON_TYPE']=='PEDESTRIAN']['TOTAL_FATAL'],c='m',marker="o",ls='--', linewidth=2,label='PEDESTRIAN')

ax.set_xlabel("YEAR")
ax.set_ylabel("Percent")
ax.set_ylim(0,100)
plt.legend(loc=1)
plt.title("Fatalities Percentage Distribution over year",fontsize=20)

space = 5
va = 'bottom'
 
for i in range(0,3):
    line = ax.lines[i]
    for x_value, y_value in zip(line.get_xdata(), line.get_ydata()):
        label = "{:.2f}".format(y_value)
        ax.annotate(label,(x_value, y_value), xytext=(0, space), 
            textcoords="offset points", ha='center', va=va)

plt.show()

In [ ]:
# from matplotlib import rc
 
# plt.subplots(figsize=(18,12))
    
# y-axis in bold
#rc('font', weight='bold')
 
# Values of each group
#bars1 = [94, 374, 370, 229]
#bars2 = [15, 56, 131, 70]
#bars3 = [3, 15, 19, 12]
 
# Heights of bars1 + bars2
#bars = np.add(bars1, bars2).tolist()
 
# The position of the bars on the x-axis
#r = [0,2,4,6]
 
# Names of group and bar width
#names = ['2017','2018','2019','2020']
#barWidth = 1
 
# Create brown bars
#plt.bar(r, bars1, color='#7f6d5f', edgecolor='white', width=barWidth, label='PEOPLE IN VEHICLES')
# Create green bars (middle), on top of the firs ones
#plt.bar(r, bars2, bottom=bars1, color='#557f2d', edgecolor='white', width=barWidth, label='PEOPLE WALKING')
# Create green bars (top)
#plt.bar(r, bars3, bottom=bars, color='#2d7f5e', edgecolor='white', width=barWidth, label='PEOPLE BICYCLING')
 
# Custom X axis
#plt.xticks(r, names, fontweight='bold', fontsize=18)
#plt.yticks(fontweight='bold', fontsize=18)
#plt.xlabel("Year", fontsize=20)
#plt.ylabel("Number of Serious Injuries and Fatalities", fontsize=20)
#plt.legend(loc='upper left', frameon=False, fontsize=16)
 
# Show graphic
#plt.show()

## Vulnerability of Roadway Users

- People walking are **7.5 times more likely** to be killed or seriously injured. 

- People bicycling are **6.2 times more likely** to be killed or seriously injured.

In [ ]:
n_driver_inj=train_new.loc[(train_new["INJURY"]=='INJURED')&(train_new["PERSON_TYPE"]=="DRIVER"),:].shape[0]
n_driver_acc=train_new.loc[train_new["PERSON_TYPE"]=="DRIVER", :].shape[0]
p_driver = round(n_driver_inj/n_driver_acc, 2)
print('Probability of getting seriously injured or killed in an accident for', 
      '\033[1m', 'a driver', '\033[0m', 'is', '\033[1m', p_driver, '%.', '\033[0m')

print()

n_ped_inj=train_new.loc[(train_new["INJURY"]=='INJURED')&(train_new["PERSON_TYPE"]=="PEDESTRIAN"),:].shape[0]
n_ped_acc=train_new.loc[train_new["PERSON_TYPE"]=="PEDESTRIAN", :].shape[0]
p_ped = round(n_ped_inj/n_ped_acc, 2)
print('Probability of getting seriously injured or killed in an accident for', 
      '\033[1m', 'a pedestrian', '\033[0m', 'is', '\033[1m', p_ped, '%.', '\033[0m')

print()

n_bcycle_inj=train_new.loc[(train_new["INJURY"]=='INJURED')&(train_new["PERSON_TYPE"]=="BICYCLE"),:].shape[0]
n_bcycle_acc=train_new.loc[train_new["PERSON_TYPE"]=="BICYCLE", :].shape[0]
p_bcycle = round(n_bcycle_inj/n_bcycle_acc, 2)
print('Probability of getting seriously injured or killed in an accident for', 
      '\033[1m', 'a cycler', '\033[0m', 'is', '\033[1m', p_bcycle, '%.', '\033[0m')

print()

print('People walking are', '\033[1m',  round(p_ped/p_driver, 2), 'times more likely', '\033[0m', 
      'to be killed or seriously injured!')

print()

print('People bicycling are', '\033[1m',  round(p_bcycle/p_driver, 2), 'times more likely', '\033[0m', 
      'to be killed or seriously injured!')

In [ ]:
print('\033[1m', 'Probability of getting injured in an accident by Person Type', '\033[0m')
round((train_new[train_new['INJURY']=='INJURED']['PERSON_TYPE'].value_counts() / train_new.shape[train_new['PERSON_TYPE']=='INJURED'] * 100),2)

In [ ]:
print('\033[1m', 'Number of Serious Injuries and Fatalities by Person Type', '\033[0m')
train_new.groupby(['PERSON_TYPE'])['TOTAL_FATAL'].sum()

In [ ]:
#train_new.shape[0]  #53178
#train_new.loc(train_new['PERSON_TYPE'=='BIBCYCLE'], )['TOTAL_FATAL'].sum()
print(train_new.TOTAL_FATAL.sum())
train_new.loc[train['PERSON_TYPE'] == 'BICYCLE', 'TOTAL_FATAL'].sum() 

In [ ]:
print(train_new.INJURIES_TOTAL.sum())

print('\033[1m', 'Number of Serious Injuries and Fatalities by Person Type', '\033[0m')
train_new.groupby(['PERSON_TYPE'])['INJURIES_TOTAL'].sum()



In [ ]:
print('\033[1m', 'Number of Minor Injuries by Person Type', '\033[0m')
minor_injuries = train_new.groupby(['PERSON_TYPE'])['INJURIES_TOTAL'].sum() - train_new.groupby(['PERSON_TYPE'])['TOTAL_FATAL'].sum()
minor_injuries.sort_values(ascending=False)

In [ ]:
# Pie chart
labels = ['SEVERE INJURIES', 'MINOR INJURIES']
sizes = [1067, 8602]
#colors
colors = ['#ff9999','#66b3ff']


fig1, ax1 = plt.subplots(figsize=(5,5))
ax1.pie(sizes, colors = colors, labels=labels, autopct='%1.1f%%', startangle=90)
#draw circle
centre_circle = plt.Circle((0,0),0.70,fc='white')
fig = plt.gcf()
fig.gca().add_artist(centre_circle)

plt.title('PEOPLE IN VEHICLES', fontsize=15)


# Equal aspect ratio ensures that pie is drawn as a circle
ax1.axis('equal')  
plt.tight_layout()
plt.show()

In [ ]:
# Pie chart
labels = ['SEVERE INJURIES', 'MINOR INJURIES']
sizes = [272, 862]
#colors
colors = ['#ff9999','#66b3ff']


fig1, ax1 = plt.subplots(figsize=(6,6))
ax1.pie(sizes, colors = colors, labels=labels, autopct='%1.1f%%', startangle=90)
#draw circle
centre_circle = plt.Circle((0,0),0.70,fc='white')
fig = plt.gcf()
fig.gca().add_artist(centre_circle)

plt.title('PEDESTRIAN', fontsize=15)


# Equal aspect ratio ensures that pie is drawn as a circle
ax1.axis('equal')  
plt.tight_layout()
plt.show()

In [ ]:
# Pie chart
labels = ['SEVERE INJURIES', 'MINOR INJURIES']
sizes = [49, 249]
#colors
colors = ['#ff9999','#66b3ff']


fig1, ax1 = plt.subplots(figsize=(5.5, 5.5))
ax1.pie(sizes, colors = colors, labels=labels, autopct='%1.1f%%', startangle=90)
#draw circle
centre_circle = plt.Circle((0,0),0.70,fc='white')
fig = plt.gcf()
fig.gca().add_artist(centre_circle)

plt.title('PEOPLE BYCYCLING', fontsize=15)


# Equal aspect ratio ensures that pie is drawn as a circle
ax1.axis('equal')  
plt.tight_layout()
plt.show()

## More Impacted Communities

In [ ]:
# Create new table to store lat and long information

train_new['geom'] = train_new['LATITUDE'].map(str) + ',' + train_new['LONGITUDE'].map(str)
train_new['geom'][0]

In [ ]:
!pip install geopandas
!pip install geopy
import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [ ]:
# timeout to prevent timeout errors
locator = Nominatim(user_agent = 'myGeocoder', timeout=10)

rgeocode = RateLimiter(locator.reverse, min_delay_seconds=0.0000000001)

In [ ]:
!pip install tqdm
import tqdm
from tqdm import tqdm, tqdm_notebook

tqdm_notebook().pandas()

#to much time consuming (7+ hours)
#train_new['address'] = train_new['geom'].progress_apply(rgeocode)
train_new.head()

In [ ]:
import folium
from folium.plugins import MarkerCluster

map3 = folium.Map(location = [41.864073,-87.706819], zoom_start = 10.5)

marker_cluster = folium.plugins.MarkerCluster().add_to(map3)

locations = train_new[['LATITUDE', 'LONGITUDE']]
locationlist = locations.values.tolist()

for point in range(0, len(locationlist)):
    folium.Marker(locationlist[point], icon = folium.Icon(color = 'darkblue', icon_color='white', icon='male', 
                                                          angle=0, prefix='fa')).add_to(marker_cluster)
map3

**These 8 ommunities collectively account for 60% of fatal crashes!**

- **South Deering:** 22% Fatality


- **North Lawndale:** 7% Fatality


- **West Elsdon:** 7% Fatality


- **Pullman:** 7% Fatality


- **West Englewood:** 7% Fatality


- **Lake View:** 5% Fatality


- **Garfield Ridge:** 5% Fatality


- **Hegewisch:** 5% Fatality


Collectively account for more than 60% fata crashes, but only 20% of Chicago’s area and 25% of the city’s population. (Fact check needed)